In [79]:
from collections import deque
from typing import Optional
import time

## Exercise 1: BFS Implementation

In [80]:
def bfs(graph, start, goal):
    queue = deque()
    queue.append([start])

    visited = set()

    nodes_expanded = 0 

    while queue:
        path = queue.popleft()
        current_node = path[-1]

        if current_node == goal:
            return path, nodes_expanded
        
        if current_node not in visited:
            visited.add(current_node)
            nodes_expanded += 1

            for neighbor in graph[current_node]:
                new_path  = path + [neighbor]
                queue.append(new_path)
    return None, nodes_expanded

In [81]:
maze = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E', 'G'],
    'G': []
}

start_node = 'A'
goal_node = 'G'

In [82]:
path_bfs, nodes_bfs = bfs(maze, start_node, goal_node)
print('Using BFS')
print(path_bfs)
print(nodes_bfs)

Using BFS
['A', 'C', 'F', 'G']
6


## Exercise 2: Recursive DFS Implementation

In [83]:
def dfs(maze, current, goal, visited=None, path=None, nodes_expanded=0):
    if visited is None:
        visited = set()
    if path is None:
        path = [current]
    
    if current == goal:
        return path, nodes_expanded
    
    if current not in visited:
        visited.add(current)
        nodes_expanded += 1
        
        for neighbor in maze[current]:
            if neighbor not in visited:
                new_path = path + [neighbor]
                result, expanded = dfs(maze, neighbor, goal, visited, new_path, nodes_expanded)
                if result is not None:
                    return result, expanded
    
    return None, nodes_expanded

In [84]:
path_dfs, nodes_dfs = dfs(maze, start_node, goal_node)
print('Using DFS')
print(path_dfs)
print(nodes_dfs)

Using DFS
['A', 'B', 'E', 'F', 'G']
4


## Exercise 3: 8 Puzzle using BFS

In [85]:
class PuzzleState:
    def __init__(self, board: list[list[int]], blank_pos: tuple[int, int], moves: list[str] | None = None):
        self.board = board 
        self.blank_pos = blank_pos 
        self.moves = moves if moves else []

    def __eq__(self, other):
        return self.board == other.board
    
    def __hash__(self):
        return hash(tuple(tuple(row) for row in self.board))
    
    def __str__(self):
        return '\n'.join([' '.join(map(str, row)) for row in self.board])
    
def get_valid_actions(state: PuzzleState)->list[str]:
    valid_moves = []
    row, col = state.blank_pos 

    if row < 2:
        valid_moves.append('up')
    if row > 0:
        valid_moves.append('down')
    if col < 2:
        valid_moves.append('left')
    if col > 0:
        valid_moves.append('right')
    
    return valid_moves 

def apply_action(state: PuzzleState, action)->PuzzleState:
    row, col = state.blank_pos

    new_board = [row[:] for row in state.board]

    if action == 'up':
        nr, nc = row+1, col 
    elif action == 'down':
        nr, nc = row-1, col 
    elif action == 'left':
        nr, nc = row, col+1 
    else:
        nr, nc = row, col-1 

    new_board[row][col], new_board[nr][nc] = new_board[nr][nc], new_board[row][col] 
    new_moves = state.moves + [action]
    return PuzzleState(new_board, (nr, nc), new_moves)

def bfs_8puzzle(initial_state: PuzzleState, goal_state: PuzzleState)->tuple[Optional[list[str]], int, Optional[PuzzleState]]:
    queue = deque([initial_state])
    visited = set()
    visited.add(initial_state)
    states_expanded = 0 

    while queue: 
        current_state = queue.popleft() 
        states_expanded += 1 

        if current_state == goal_state:
            return current_state.moves, states_expanded, current_state
        
        for action in get_valid_actions(current_state):
            new_state = apply_action(current_state, action)
            if new_state not in visited:
                visited.add(new_state)
                queue.append(new_state)

    return None, states_expanded, None 

In [86]:
init_board = [
    [1, 2, 3],
    [4, 0, 5],
    [6, 7, 8]
]

goal_board = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

init_state = PuzzleState(init_board, (1, 1))
goal_state = PuzzleState(goal_board, (2, 2))

print("Initial State:")
print(init_state)
print("Goal State:")
print(goal_state)

moves, exp, final_state = bfs_8puzzle(init_state, goal_state)

if moves:
    print(f"\nSolution found!")
    print(f"Number of moves: {len(moves)}")
    print(f"States expanded: {exp}")
    print(f"\nSequence of moves: {moves}")
    
    print("\n" + "="*50)
    print("Step-by-step solution:")
    print("\nInitial:")
    print(init_state)
    
    current = init_state
    for i, move in enumerate(moves, 1):
        current = apply_action(current, move)
        print(f"\nStep {i}: Move {move}")
        print(current)
else:
    print("No solution found!")
    print(f"States expanded: {exp}")

Initial State:
1 2 3
4 0 5
6 7 8
Goal State:
1 2 3
4 5 6
7 8 0

Solution found!
Number of moves: 14
States expanded: 5162

Sequence of moves: ['left', 'up', 'right', 'right', 'down', 'left', 'up', 'left', 'down', 'right', 'right', 'up', 'left', 'left']

Step-by-step solution:

Initial:
1 2 3
4 0 5
6 7 8

Step 1: Move left
1 2 3
4 5 0
6 7 8

Step 2: Move up
1 2 3
4 5 8
6 7 0

Step 3: Move right
1 2 3
4 5 8
6 0 7

Step 4: Move right
1 2 3
4 5 8
0 6 7

Step 5: Move down
1 2 3
0 5 8
4 6 7

Step 6: Move left
1 2 3
5 0 8
4 6 7

Step 7: Move up
1 2 3
5 6 8
4 0 7

Step 8: Move left
1 2 3
5 6 8
4 7 0

Step 9: Move down
1 2 3
5 6 0
4 7 8

Step 10: Move right
1 2 3
5 0 6
4 7 8

Step 11: Move right
1 2 3
0 5 6
4 7 8

Step 12: Move up
1 2 3
4 5 6
0 7 8

Step 13: Move left
1 2 3
4 5 6
7 0 8

Step 14: Move left
1 2 3
4 5 6
7 8 0


## Exercise 4: Time and Memory Comparison of BFS and DFS

In [87]:
def bfs_timed(graph, start, goal):
    queue = deque()
    queue.append([start])
    visited = set()
    nodes_expanded = 0
    max_queue_size = 0
    
    start_time = time.time()
    
    while queue:
        max_queue_size = max(max_queue_size, len(queue))
        
        path = queue.popleft()
        current_node = path[-1]
        
        if current_node == goal:
            end_time = time.time()
            return {
                'path': path,
                'nodes_expanded': nodes_expanded,
                'path_length': len(path) - 1,
                'execution_time': end_time - start_time,
                'max_memory': max_queue_size,
                'solution_found': True
            }
        
        if current_node not in visited:
            visited.add(current_node)
            nodes_expanded += 1
            
            for neighbor in graph[current_node]:
                new_path = path + [neighbor]
                queue.append(new_path)
    
    end_time = time.time()
    return None 

In [88]:
def dfs_timed(graph, start, goal):
    stack = deque()
    stack.append([start])
    visited = set()
    nodes_expanded = 0
    max_stack_size = 0
    
    start_time = time.time()
    
    while stack:
        max_stack_size = max(max_stack_size, len(stack))
        
        path = stack.pop() 
        current_node = path[-1]
        
        if current_node == goal:
            end_time = time.time()
            return {
                'path': path,
                'nodes_expanded': nodes_expanded,
                'path_length': len(path) - 1, 
                'execution_time': end_time - start_time,
                'max_memory': max_stack_size,
                'solution_found': True
            }
        
        if current_node not in visited:
            visited.add(current_node)
            nodes_expanded += 1
            
            for neighbor in graph[current_node]:
                new_path = path + [neighbor]
                stack.append(new_path)
    
    end_time = time.time()
    return None 

In [89]:
maze1 = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E', 'G'],
    'G': []
}

print("==================================")
print("TEST CASE 1: Simple Maze Graph")
print(maze1)
print("==================================")

print(bfs_timed(maze1, 'A', 'G'))
print(dfs_timed(maze1, 'A', 'G'))

TEST CASE 1: Simple Maze Graph
{'A': ['B', 'C'], 'B': ['A', 'D', 'E'], 'C': ['A', 'F'], 'D': ['B'], 'E': ['B', 'F'], 'F': ['C', 'E', 'G'], 'G': []}
{'path': ['A', 'C', 'F', 'G'], 'nodes_expanded': 6, 'path_length': 3, 'execution_time': 1.1205673217773438e-05, 'max_memory': 6, 'solution_found': True}
{'path': ['A', 'C', 'F', 'G'], 'nodes_expanded': 3, 'path_length': 3, 'execution_time': 3.814697265625e-06, 'max_memory': 5, 'solution_found': True}


In [90]:
maze2 = {
    'A': ['B', 'C'],
    'B': ['D'],
    'C': ['E'],
    'D': ['F'],
    'E': ['F'],
    'F': ['G'],
    'G': []
}

print("==================================")
print("TEST CASE 2: Graph with Multiple Path Lengths")
print(maze2)
print("==================================")

print(bfs_timed(maze2, 'A', 'G'))
print(dfs_timed(maze2, 'A', 'G'))

TEST CASE 2: Graph with Multiple Path Lengths
{'A': ['B', 'C'], 'B': ['D'], 'C': ['E'], 'D': ['F'], 'E': ['F'], 'F': ['G'], 'G': []}
{'path': ['A', 'B', 'D', 'F', 'G'], 'nodes_expanded': 6, 'path_length': 4, 'execution_time': 5.7220458984375e-06, 'max_memory': 2, 'solution_found': True}
{'path': ['A', 'C', 'E', 'F', 'G'], 'nodes_expanded': 4, 'path_length': 4, 'execution_time': 3.0994415283203125e-06, 'max_memory': 2, 'solution_found': True}


In [91]:
maze3 = {
    'A': ['B', 'C', 'D'],
    'B': ['E', 'F'],
    'C': ['G', 'H'],
    'D': ['I', 'J'],
    'E': ['K'],
    'F': ['K'],
    'G': ['K'],
    'H': ['K'],
    'I': ['K'],
    'J': ['K'],
    'K': []
}

print("==================================")
print("TEST CASE 3: Deeper Graph (Memory Comparison)")
print(maze3)
print("==================================")

print(bfs_timed(maze3, 'A', 'G'))
print(dfs_timed(maze3, 'A', 'G'))


TEST CASE 3: Deeper Graph (Memory Comparison)
{'A': ['B', 'C', 'D'], 'B': ['E', 'F'], 'C': ['G', 'H'], 'D': ['I', 'J'], 'E': ['K'], 'F': ['K'], 'G': ['K'], 'H': ['K'], 'I': ['K'], 'J': ['K'], 'K': []}
{'path': ['A', 'C', 'G'], 'nodes_expanded': 6, 'path_length': 2, 'execution_time': 7.152557373046875e-06, 'max_memory': 6, 'solution_found': True}
{'path': ['A', 'C', 'G'], 'nodes_expanded': 7, 'path_length': 2, 'execution_time': 6.4373016357421875e-06, 'max_memory': 4, 'solution_found': True}


1. Why does BFS guarantee an optimal solution while DFS does not? <br>
   Answer: BFS explores all nodes at distance k before exploring
   nodes at distance k+1. This level-by-level exploration ensures
   that when BFS finds the goal, it has found the shortest path.
   DFS goes deep first and returns the first path it finds, which
   may not be the shortest.

2. In which scenarios is DFS preferred over BFS? <br>
   Answer: DFS is preferred when:
   - Memory is limited (DFS uses less memory)
   - Solutions are deep in the tree
   - Any solution is acceptable (not necessarily shortest)
   - The graph is very wide but not very deep
   - You need to explore all paths (e.g., detecting cycles)

3. How does the branching factor affect BFS performance? <br>
   Answer: Higher branching factor means more neighbors per node.
   BFS must store ALL neighbors at each level in the queue,
   causing exponential memory growth with branching factor.
   Memory usage = O(b^d) where b=branching factor, d=depth

4. Why is BFS considered impractical for large state spaces? <br>
   Answer: BFS stores all nodes at the current level in memory.
   For large state spaces (like 8-puzzle with 181,440 states),
  this can quickly exhaust memory. The queue can grow to contain
   thousands or millions of states, each storing full path history.